In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
df=pd.read_csv("/kaggle/input/algerian-forest-fires-dataset/Algerian_forest_fires_dataset.csv")
df.head()

In [ ]:
df.info()

In [ ]:
#data cleaning
df.isna().sum()

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.loc[:122,"Region"]=0
df.loc[122:,"Region"]=1

In [ ]:
df.info()

In [ ]:
df[["Region"]]=df[["Region"]].astype(int)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df=df.dropna().reset_index(drop=True)

In [ ]:
df.isna().sum()

In [ ]:
df.iloc[[122]]

In [ ]:
df=df.drop(122).reset_index(drop=True)

In [ ]:
df.iloc[[122]]

In [ ]:
df.columns

In [ ]:
df.columns=df.columns.str.strip()

In [ ]:
df.columns

In [ ]:
df[['day', 'month', 'year', 'Temperature', 'RH', 'Ws']]=df[['day', 'month', 'year', 'Temperature', 'RH', 'Ws']].astype(int)

In [ ]:
df.info()

In [ ]:
object=[feature for feature in df.columns if df[feature].dtype=='O' ]

In [ ]:
for i in object:
    if i!="Classes":
        df[i]=df[i].astype(float)

In [ ]:
df.info()

In [ ]:
df1=df.copy()

In [ ]:
df1=df.drop(["day","month","year"],axis=1)

In [ ]:
df1.head()

In [ ]:
df1['Classes']=np.where(df1["Classes"].str.contains('not fire'), 0, 1)

In [ ]:
df1.head()

In [ ]:
df1["Classes"].value_counts()

In [ ]:
df["Classes"].value_counts()

In [ ]:
plt.style.use('seaborn')
df1.hist(bins=50,figsize=(20,15))
plt.show()

In [ ]:
percentage=df1["Classes"].value_counts(normalize=True)*100

In [ ]:
labels=["fire","Not fire"]
plt.figure(figsize=(12,7))
plt.pie(percentage,labels=labels,autopct='%1.1f%%')
plt.show()

In [ ]:
df1.corr()

In [ ]:
sns.heatmap(df1.corr())

In [ ]:
sns.boxplot(df1['FWI'],color='grey')

In [ ]:
df['Classes']=np.where(df['Classes'].str.contains('not fire'),'not fire','fire')

In [ ]:
temp=df.loc[df['Region']==1]
plt.subplots(figsize=(13,6))
sns.set_style('whitegrid')
sns.countplot(x='month',hue='Classes',data=df)
plt.title("Sidi-Bel")

In [ ]:
temp1=df.loc[df['Region']==0]
plt.subplots(figsize=(12,6))
sns.set_style('whitegrid')
sns.countplot(x='month',hue='Classes',data=df)
plt.title("Bejaia Region")

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
X=df1.drop('FWI',axis=1)
y=df1['FWI']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [ ]:
sns.heatmap(df1.corr(),annot=True)

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: 
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_fet=correlation(X_train,0.85)

In [ ]:
X_train.drop(corr_fet,axis=1,inplace=True)
X_test.drop(corr_fet,axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:

plt.subplots(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.boxplot(data=X_train)
plt.title('X_train Before Scaling')
plt.subplot(1, 2, 2)
sns.boxplot(data=X_train_scaled)
plt.title('X_train After Scaling')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,r2_score
linear=LinearRegression()
linear.fit(X_train_scaled,y_train)
y_pred=linear.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(mae)
print(r2)
plt.scatter(y_test,y_pred)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error,r2_score
lasso=Lasso()
lasso.fit(X_train_scaled,y_train)
y_pred=lasso.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(mae)
print(r2)
plt.scatter(y_test,y_pred)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error,r2_score
ridge=Ridge()
ridge.fit(X_train_scaled,y_train)
y_pred=linear.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(mae)
print(r2)
plt.scatter(y_test,y_pred)

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error,r2_score
elastic=ElasticNet()
elastic.fit(X_train_scaled,y_train)
y_pred=linear.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(mae)
print(r2)
plt.scatter(y_test,y_pred)

In [ ]:
from sklearn.linear_model import LassoCV
lassocv=LassoCV(cv=5)
lassocv.fit(X_train_scaled,y_train)



In [ ]:
lassocv.alpha_

In [ ]:
lassocv.alphas_

In [ ]:
y_pred=lassocv.predict(X_test_scaled)
plt.scatter(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)

In [ ]:
from sklearn.linear_model import RidgeCV
ridgecv=RidgeCV(cv=5)
ridgecv.fit(X_train_scaled,y_train)
y_pred=ridgecv.predict(X_test_scaled)
plt.scatter(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)

In [ ]:
from sklearn.linear_model import ElasticNetCV
elasticcv=ElasticNetCV(cv=5)
elasticcv.fit(X_train_scaled,y_train)
y_pred=elasticcv.predict(X_test_scaled)
plt.scatter(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)